# Utility functions

In [1]:
# default_exp utils
# export
import torch
from torch import nn, Tensor
import re
import numpy as np
from fastcore.dispatch import retain_type

In [ ]:
To Do

- [ ] Add Documentation to each function, move Docstring in Markdowncell below
- [ ] Add permalink to fastai function

## Adapted from fastai

I often use fastai to build and train models. However, the vision module of fastai is designed for 2d images. To work with 3d Modules most functions need small adjustments. Also, I wanted PyTorch as the only requirement for this repository. Therefore, I decided to transcribe the fastai functions and change them as needed. Still >90% of the code is original fastai code and I try to appropriate credit. 

### Layer manipulation

In [2]:
# export
def _get_first_layer(model):
    """ Access first layer of a model
        Copied from: https://github.com/fastai/fastai/blob/ecb67b8a3d322efeec1e3c37faa10025e5d22c49/fastai/vision/learner.py#L25
        
        Args: 
            model (nn.Module): A PyTorch model
            
        Returns: 
            child (nn.Module): The first module of the model
            parent (nn.Sequential, None): The first layer of the model
            name (str): The name of the first module

    """
    child, parent, name = model, None, None  # child, parent, name
    for name in next(model.named_parameters())[0].split('.')[:-1]:
        parent, child = child,getattr(child, name)
    return child, parent, name

In [3]:
# export
def _load_pretrained_weights(new_layer, previous_layer):
    """
    Load pretrained weights based on number of input channels. 
    Adapted from https://github.com/fastai/fastai/blob/ecb67b8a3d322efeec1e3c37faa10025e5d22c49/fastai/vision/learner.py#L33
    Compared to the fastai function, this function can handle various number of input channels for the `previous_layer` and 
    inits new channels in the `new_layer` with normal distribution not zero
    
    Args: 
        new_layer (nn.ConvNd): The new layer to transfer weights to 
        previous layer (nn.ConvNd): The old layer to copy weights from
        
    Returns: 
        None (updates layer in place)
    """
    
    new_in = getattr(new_layer, 'in_channels')
    prev_in = getattr(previous_layer, 'in_channels')
    if new_in==1:
        # we take the sum
        new_layer.weight.data = previous_layer.weight.data.sum(dim=1, keepdim=True)
    elif new_in < prev_in:
        # we copy weights of the first n-channels from previous_layer
        # then add the prozetual decrease in channel size
        new_layer.weight.data = previous_layer.weight.data[:,:new_in] * prev_in/new_in
    else:
        # keep channels weights and init the other with normal distribution
        new_layer.weight.data[:,:prev_in] = previous_layer.weight.data
        mean, std = torch.mean(previous_layer.weight.data), torch.std(previous_layer.weight.data)
        new_layer.weight.data[:,prev_in:] = nn.init.normal_(mean, std)

In [4]:
# export
def _update_first_layer(model, n_in, pretrained=True):
    """ Change first layer based on number of input channels
        Adapted from fastai implementation: https://github.com/fastai/fastai/blob/ecb67b8a3d322efeec1e3c37faa10025e5d22c49/fastai/vision/learner.py#L48
        
        Args: 
            model (nn.Module): A PyTorch Model with first layer beeing a nn.Conv3d
            n_in (int): Number of input channels
            pretrained (bool): Wether to load pretrained weigths for the model (if available)
            
        Returns: 
            None (updated layer in place)
    """
    first_layer, parent, name = _get_first_layer(model)
    assert isinstance(first_layer, nn.Conv3d), f'Change of input channels only supported with Conv3d, found {first_layer.__class__.__name__}'
    params = {attr:getattr(first_layer, attr) for attr in 'out_channels kernel_size stride padding dilation groups padding_mode'.split()}
    params['bias'] = getattr(first_layer, 'bias') is not None
    params['in_channels'] = n_in
    new_layer = nn.Conv3d(**params)
    if pretrained:
        _load_pretrained_weights(new_layer, first_layer)
    setattr(parent, name, new_layer)

In [5]:
# export
def _has_pool_type(module):
    """ Return `True` if `module` is a pooling layer or has one in its children
        Nearly identical to: https://github.com/fastai/fastai/blob/ecb67b8a3d322efeec1e3c37faa10025e5d22c49/fastai/vision/learner.py#L17
        
        Args: 
            module (nn.Module): A PyTorch nn.Module or nn.Sequential
            
        Returns: 
            bool
    """
    for layer in [module, *module.children()]:
        if re.search(r'Pool[123]d$', layer.__class__.__name__): return True
    return False

In [6]:
# export
def create_body(arch, n_in, cut=None, pretrained=True):
    """ Cut off the body of a typically pretrained `arch` as determined by `cut`
        Identical to: https://github.com/fastai/fastai/blob/ecb67b8a3d322efeec1e3c37faa10025e5d22c49/fastai/vision/learner.py#L63
        
        Args: 
            arch (callable): Function to construct the model
            n_in (int): Number of input channels
            cut (None, int, callable): If None, the position to cut of the body is determined automatically.
                                       If int, the model is cut at the specified position. 
                                       If callabe, this function is used to cut the model
        
        Returns: 
            The body of the model
    """
    model = arch(pretrained=pretrained)
    _update_first_layer(model, n_in, pretrained)
    #cut = ifnone(cut, cnn_config(arch)['cut'])
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if _has_pool_type(o))
    if isinstance(cut, int): 
        return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): 
        return cut(model)
    else: 
        raise NamedError("cut must be either integer or a function")

In [7]:
# export
def in_channels(module):
    "Get the number of input_channels in the first weight layer in `module`."
    return _get_first_layer(module)[0].weight.shape[1]

In [8]:
# export
def _get_sz_change_idxs(sizes):
    "Get the indexes of the layers where the size of the activation changes."
    feature_szs = [size[-1] for size in sizes]
    sz_chg_idxs = list(np.where(np.array(feature_szs[:-1]) != np.array(feature_szs[1:]))[0])
    return sz_chg_idxs

In [9]:
# export
def first(x, f=None, negate=False, **kwargs):
    "First element of `x`, optionally filtered by `f`, or None if missing"
    x = iter(x)
    if f: x = filter_ex(x, f=f, negate=negate, gen=True, **kwargs)
    return next(x, None)

In [10]:
# export
def dummy_eval(m, size=(8,64,64)):
    "Evaluate `m` on a dummy input of a certain `size`. Same as fastai func"
    ch_in = in_channels(m)
    x = first(m.parameters()).new(1, ch_in, *size).requires_grad_(False).uniform_(-1.,1.)
    with torch.no_grad(): return m.eval()(x)

In [11]:
# export
def model_sizes(m, size=(8,64,64)):
    "Pass a dummy input through the model `m` to get the various sizes of activations. same as fastai func"
    with hook_outputs(m) as hooks:
        _ = dummy_eval(m, size=size)
        return [o.stored.shape for o in hooks]

### Hooks

In [12]:
# export
class Hook():
    "Create a hook on `m` with `hook_func`."
    def __init__(self, m, hook_func, is_forward=True, detach=True, cpu=False, gather=False):
        self.hook_func, self.detach, self.cpu, self.gather = hook_func, detach, cpu, gather
        f = m.register_forward_hook if is_forward else m.register_backward_hook
        self.hook = f(self.hook_fn)
        self.stored,self.removed = None,False

    def hook_fn(self, module, input, output):
        "Applies `hook_func` to `module`, `input`, `output`."
        if self.detach:
            input,output = to_detach(input, cpu=self.cpu, gather=self.gather),to_detach(output, cpu=self.cpu, gather=self.gather)
        self.stored = self.hook_func(module, input, output)

    def remove(self):
        "Remove the hook from the model."
        if not self.removed:
            self.hook.remove()
            self.removed=True

    def __enter__(self, *args): return self
    def __exit__(self, *args): self.remove()

    _docs = dict(__enter__="Register the hook",
                 __exit__="Remove the hook")

In [13]:
# export
def _hook_inner(m,i,o): return o if isinstance(o,Tensor) or is_listy(o) else list(o)

In [14]:
# export
def hook_output(module, detach=True, cpu=False, grad=False):
    "Return a `Hook` that stores activations of `module` in `self.stored`"
    return Hook(module, _hook_inner, detach=detach, cpu=cpu, is_forward=not grad)

In [15]:
# export
class Hooks():
    "Create several hooks on the modules in `ms` with `hook_func`."
    def __init__(self, ms, hook_func, is_forward=True, detach=True, cpu=False):
        self.hooks = [Hook(m, hook_func, is_forward, detach, cpu) for m in ms]

    def __getitem__(self,i): return self.hooks[i]
    def __len__(self):       return len(self.hooks)
    def __iter__(self):      return iter(self.hooks)
    @property
    def stored(self):        return L(o.stored for o in self)

    def remove(self):
        "Remove the hooks from the model."
        for h in self.hooks: h.remove()

    def __enter__(self, *args): return self
    def __exit__ (self, *args): self.remove()

    _docs = dict(stored = "The states saved in each hook.",
                 __enter__="Register the hooks",
                 __exit__="Remove the hooks")

In [16]:
# export 
def hook_outputs(modules, detach=True, cpu=False, grad=False):
    "Return `Hooks` that store activations of all `modules` in `self.stored`"
    return Hooks(modules, _hook_inner, detach=detach, cpu=cpu, is_forward=not grad)

In [17]:
# export
def maybe_gather(x, axis=0):
    "Gather copies of `x` on `axis` (if training is distributed)"
    if num_distrib()<=1: return x
    ndim = x.ndim
    res = [x.new_zeros(*x.shape if ndim > 0 else (1,)) for _ in range(num_distrib())]
    torch.distributed.all_gather(res, x.contiguous() if ndim > 0 else x[None])
    return torch.cat(res, dim=axis) if ndim > 0 else torch.cat(res, dim=axis).mean()

In [18]:
# export
def to_detach(b, cpu=True, gather=True):
    "Recursively detach lists of tensors in `b `; put them on the CPU if `cpu=True`."
    def _inner(x, cpu=True, gather=True):
        if not isinstance(x,Tensor): return x
        x = x.detach()
        if gather: x = maybe_gather(x)
        return x.cpu() if cpu else x
    return apply(_inner, b, cpu=cpu, gather=gather)

In [19]:
# export
def apply(func, x, *args, **kwargs):
    "Apply `func` recursively to `x`, passing on args"
    if isinstance(x, (list, tuple)): return type(x)([apply(func, o, *args, **kwargs) for o in x])
    if isinstance(x,dict):  return {k: apply(func, v, *args, **kwargs) for k,v in x.items()}
    res = func(x, *args, **kwargs)
    return res if x is None else retain_type(res, x)

In [20]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_unet.ipynb.
Converted index.ipynb.
